In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import xgboost as xgb

from sklearn.neighbors import KNeighborsClassifier

In [2]:
data = 'TRs_annotated_cleaned.csv'
df = pd.read_csv(data)

FileNotFoundError: [Errno 2] No such file or directory: 'TRs_annotated_cleaned.csv'

In [ ]:
col_names = df.columns

In [ ]:
col_names = df.columns
categorical = [var for var in df.columns if df[var].dtype=='O']
numerical = [var for var in df.columns if df[var].dtype!='O']

In [ ]:
print(numerical)
print(categorical)

['gene_distance', 'gerp', 'TAD', 'eSTR', 'opReg', 'promoter', 'UTR_3', 'UTR_5', 'loeuf', 'pLi', 'RAD21', 'SMC3', 'per_g', 'per_c', 'per_a', 'per_t', 'gc_content', 'eSh0', 'eSh1', 'eSh2', 'eSh3', 'eSh4', 'eSh5', 'eTr0', 'eTr2', 'eTr3', 'eTr4', 'eTr5', 'eH', 'eW', 'eS6', 'eS', 'J', 'eX0', 'eX1R', 'eX2', 'eX3', 'eX4', 'eX5', 'tissue_simple_No_expression', 'tissue_simple_Nervous_System', 'tissue_simple_Other', 'region_intergenic', 'region_intron', 'region_exon', 'location_First', 'location_Middle', 'location_Last', 'label']
['id', 'location', 'region', 'gene_type', 'tissue_simple']


In [ ]:
# changing categorial variables into one-hot
# ['id', 'location', 'region', 'gene_type', 'tissue_simple']
df.drop(['id', 'location', 'region', 'tissue_simple'], axis=1, inplace=True)

one_hot_gene_type = pd.get_dummies(df.gene_type, prefix="gene_type", drop_first=True, dtype=int)
new_df = pd.concat([df[numerical], one_hot_gene_type], axis=1)
new_df.drop(['gene_type_intergenic'], axis=1, inplace=True)

In [ ]:
new_df.head()

,gene_distance,gerp,TAD,eSTR,opReg,promoter,UTR_3,UTR_5,loeuf,pLi,...,location_Last,label,gene_type_Unknown,gene_type_antisense,gene_type_lincRNA,gene_type_polymorphic_pseudogene,gene_type_processed_transcript,gene_type_protein_coding,gene_type_pseudogene,gene_type_sense_overlapping
0,6407,2.790,0,0,0,0,0,0,9.00,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0,-0.015,0,0,1,0,0,0,0.70,0.0,...,0,0,0,0,0,0,0,1,0,0
2,0,0.117,0,0,1,0,0,0,0.70,0.0,...,0,1,0,0,0,0,0,1,0,0
3,0,0.113,0,0,1,0,0,0,0.85,0.0,...,0,0,0,0,0,0,0,1,0,0
4,0,2.770,0,0,0,0,0,0,0.85,0.0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
X = new_df.drop(['label'], axis=1)
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 32)

In [ ]:
# List of column names
all_columns = X.columns

# List to store column names where all values are 0 or 1
binary_columns = []

# Iterate over each column
for column in all_columns:
    unique_values = X[column].unique()
    # Check if unique values contain only 0 and 1
    if set(unique_values) == {0, 1}:
        binary_columns.append(column)

# Print column names where all values are 0 or 1
# print("Columns with only 0 or 1 values:")
for column in binary_columns:
    # print(column)
    pass

In [ ]:
numeric_columns = [elem for elem in all_columns if elem not in binary_columns]

In [ ]:
# Create a ColumnTransformer to apply different transformations to numeric and binary columns
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', StandardScaler(), numeric_columns),
        ('binary', 'passthrough', binary_columns)
    ])

# Define the pipeline with the preprocessing steps
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Fit and transform the training data
X_train_processed = pipeline.fit_transform(X_train)

# Transform the test data using the fitted pipeline
X_test_processed = pipeline.transform(X_test)

# Convert the processed data back to DataFrame (optional)
X_train_processed = pd.DataFrame(X_train_processed, columns=numeric_columns + binary_columns)
X_test_processed = pd.DataFrame(X_test_processed, columns=numeric_columns + binary_columns)

In [ ]:
X_train_processed.head()

,gene_distance,gerp,loeuf,pLi,RAD21,SMC3,per_g,per_c,per_a,per_t,...,location_Middle,location_Last,gene_type_Unknown,gene_type_antisense,gene_type_lincRNA,gene_type_polymorphic_pseudogene,gene_type_processed_transcript,gene_type_protein_coding,gene_type_pseudogene,gene_type_sense_overlapping
0,-0.423659,0.009804,-1.146006,-0.426908,-0.125087,-0.128276,0.417507,-1.182041,1.912984,-0.740486,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.805738,-0.263643,0.794504,-0.426908,-0.125087,-0.128276,-0.761202,0.778989,-0.632831,0.490393,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.423659,-0.412051,-1.160744,-0.426908,-0.125087,-0.128276,-0.761202,0.778989,0.385618,-0.740486,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,-0.423659,0.228327,-1.347425,2.597147,-0.125087,-0.128276,-0.761202,-1.182041,0.385618,2.951413,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.106366,0.143021,0.794504,-0.426908,-0.125087,-0.128276,1.596216,-1.182041,0.385618,-0.740486,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
param_grid = {
    'alpha': [0, 0.2, 0.4, 0.6, 0.8, 1.0],
    'l1_ratio': [0, 0.2, 0.4, 0.6, 0.8, 1.0]
}
elastic_net = ElasticNet(random_state=42)

grid_search = GridSearchCV(estimator=elastic_net, param_grid=param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train_processed, y_train)
best_model_EN = grid_search.best_estimator_

In [ ]:
# print params
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

y_pred_prob = best_model_EN.predict(X_test_processed)

# Compute the ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC Score:", roc_auc)

Best Parameters: {'alpha': 0, 'l1_ratio': 0}
ROC AUC Score: 0.7817782537984036


In [ ]:
y_pred_prob = best_model_EN.predict(X_train_processed)

# Compute the ROC AUC score
roc_auc = roc_auc_score(y_train, y_pred_prob)
print("ROC AUC Score:", roc_auc)

ROC AUC Score: 0.7939948720142169


In [ ]:
# Access coefficient summary information
coefficients = best_model_EN.coef_
intercept = best_model_EN.intercept_
col_names = new_df.columns
# Print the coefficients
print("Coefficients:")
for i, coef in enumerate(coefficients):
    print(f"Feature {col_names[i]}: {coef}")

# Print the intercept
print("Intercept:", intercept)

Coefficients:
Feature gene_distance: -5.203737213074537e-08
Feature gerp: -0.005309499472599852
Feature TAD: -0.008059709497305487
Feature eSTR: 0.09757944836567786
Feature opReg: 0.10123255681092293
Feature promoter: 0.008470992418469447
Feature UTR_3: 0.00746362333948054
Feature UTR_5: 0.17970851465004525
Feature loeuf: -0.0022370664567566327
Feature pLi: -0.005610545968755422
Feature RAD21: -0.0005991776531262726
Feature SMC3: -0.011648205186526105
Feature per_g: 0.0011486977170186167
Feature per_c: -0.0012585908743200183
Feature per_a: 0.002149269644704774
Feature per_t: -0.0008669630385718367
Feature gc_content: -0.00025508119889866907
Feature eSh0: 0.14970770066818995
Feature eSh1: -0.004316748445625885
Feature eSh2: -0.05094285988660473
Feature eSh3: 0.09373804380661135
Feature eSh4: 0.05977645945579682
Feature eSh5: -0.09430446999685144
Feature eTr0: -0.0008603081500927252
Feature eTr2: -0.004075247106627184
Feature eTr3: 0.09897936474303567
Feature eTr4: 0.006289822358438941
F

In [ ]:
param_grid = {
    'max_depth': [2, 5],
    'learning_rate': [0.2],
    'min_child_weight': [1],
    'subsample': [0.3, 0.5],
    'colsample_bytree': [0.3, 0.4]
}
xgb_classifier = xgb.XGBClassifier(verbosity=2, random_state=42)

grid_search = GridSearchCV(verbose=2, estimator=xgb_classifier, param_grid=param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train, y_train)
best_model_XGB = grid_search.best_estimator_

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END colsample_bytree=0.3, learning_rate=0.2, max_depth=2, min_child_weight=1, subsample=0.3; total time=   1.4s
[CV] END colsample_bytree=0.3, learning_rate=0.2, max_depth=2, min_child_weight=1, subsample=0.3; total time=   0.4s
[CV] END colsample_bytree=0.3, learning_rate=0.2, max_depth=2, min_child_weight=1, subsample=0.3; total time=   1.6s
[CV] END colsample_bytree=0.3, learning_rate=0.2, max_depth=2, min_child_weight=1, subsample=0.3; total time=   0.4s
[CV] END colsample_bytree=0.3, learning_rate=0.2, max_depth=2, min_child_weight=1, subsample=0.3; total time=   0.4s
[CV] END colsample_bytree=0.3, learning_rate=0.2, max_depth=2, min_child_weight=1, subsample=0.5; total time=   0.4s
[CV] END colsample_bytree=0.3, learning_rate=0.2, max_depth=2, min_child_weight=1, subsample=0.5; total time=   0.4s
[CV] END colsample_bytree=0.3, learning_rate=0.2, max_depth=2, min_child_weight=1, subsample=0.5; total time=   0.4s
[CV]

In [ ]:
# print params
best_params = grid_search.best_params_
print("Best Parameters:", best_params)
y_pred_prob = best_model.predict(X_test)

# Compute the ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC Score:", roc_auc)

Best Parameters: {'colsample_bytree': 0.4, 'learning_rate': 0.2, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.5}
ROC AUC Score: 0.7501426040879922


In [ ]:
y_pred_prob = best_model.predict(X_train)

# Compute the ROC AUC score
roc_auc = roc_auc_score(y_train, y_pred_prob)
print("ROC AUC Score:", roc_auc)

ROC AUC Score: 0.9216571459978918


In [ ]:
param_grid = {
    'n_estimators': [500, 1500],
}
rf_classifier = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(verbose=2, estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

In [ ]:
# print params
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

y_pred_prob = best_model.predict(X_test)

# Compute the ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC Score:", roc_auc)